In [1]:
# start up again with our simple example
from databaker.framework import *
tabs = loadxlstabs("DDexample1.xls")
tab = tabs[0]  # there is only one table
#savepreviewhtml(tabs[0])


Loading DDexample1.xls which has size 7168 bytes
Table names: ['beatles', 'stones']


In [2]:
r1 = tab.excel_ref('B3').expand(RIGHT)
r2 = tab.excel_ref('A3').fill(DOWN)
obs = tab.excel_ref('B4').expand(DOWN).expand(RIGHT).is_not_blank().is_not_whitespace()
dimensions = [ 
    HDim(tab.excel_ref('B1'), TIME, CLOSEST, ABOVE), 
    HDim(r1, "Vehicles", DIRECTLY, ABOVE), 
    HDim(r2, "Name", DIRECTLY, LEFT), 
    HDimConst("Category", "Beatles")
]
c1 = ConversionSegment(tab, dimensions, obs)

# make a second conversion segment??

In [3]:
#[type(ob.value)  for ob in obs.unordered_cells]

In [4]:
#c1.processedrows[3]

In [5]:
x, Lx = writetechnicalCSV(None, c1), LwritetechnicalCSV(None, c1)
x == Lx, len(x), len(Lx)

(False, 2745, 2757)

In [6]:
LwritetechnicalCSV("DDex.csv", c1)

writing 1 conversion segments into /home/goatchurch/sensiblecode/quickcode-ons-recipes/helpnotes/DDex.csv
conversionwrite segment size 12 table 'beatles; 


In [7]:
print(Lx)

observation,data_marking,statistical_unit_eng,statistical_unit_cym,measure_type_eng,measure_type_cym,observation_type,empty,obs_type_value,unit_multiplier,unit_of_measure_eng,unit_of_measure_cym,confidentuality,empty1,geographic_area,empty2,empty3,time_dim_item_id,time_dim_item_label_eng,time_dim_item_label_cym,time_type,empty4,statistical_population_id,statistical_population_label_eng,statistical_population_label_cym,cdid,cdiddescrip,empty5,empty6,empty7,empty8,empty9,empty10,empty11,empty12,dim_id_1,dimension_label_eng_1,dimension_label_cym_1,dim_item_id_1,dimension_item_label_eng_1,dimension_item_label_cym_1,is_total_1,is_sub_total_1,dim_id_2,dimension_label_eng_2,dimension_label_cym_2,dim_item_id_2,dimension_item_label_eng_2,dimension_item_label_cym_2,is_total_2,is_sub_total_2,dim_id_3,dimension_label_eng_3,dimension_label_cym_3,dim_item_id_3,dimension_item_label_eng_3,dimension_item_label_cym_3,is_total_3,is_sub_total_3
2.0,,,,,,,,,,,,,,,,,2014,2014,,Year,,,,,,,,,,,,,0,,Vehicles,V

In [12]:
readtechnicalCSV(Lx)


{0: [{'Name': 'John',
   -1: 'Year',
   -9: '2.0',
   'Vehicles': 'Cars',
   -2: '2014',
   'Category': 'Beatles'},
  {'Name': 'John',
   -1: 'Year',
   -9: '2.0',
   'Vehicles': 'Planes',
   -2: '2014',
   'Category': 'Beatles'},
  {'Name': 'John',
   -1: 'Year',
   -9: '1.0',
   'Vehicles': 'Trains',
   -2: '2014',
   'Category': 'Beatles'},
  {'Name': 'Paul',
   -1: 'Year',
   -9: '3',
   -8: 'xxx',
   'Vehicles': 'Cars',
   -2: '2014',
   'Category': 'Beatles'},
  {'Name': 'Paul',
   -1: 'Year',
   -9: '3.0',
   'Vehicles': 'Planes',
   -2: '2014',
   'Category': 'Beatles'},
  {'Name': 'Paul',
   -1: 'Year',
   -9: '2.0',
   'Vehicles': 'Trains',
   -2: '2014',
   'Category': 'Beatles'},
  {'Name': 'Ringo',
   -1: 'Year',
   -9: '4.0',
   'Vehicles': 'Cars',
   -2: '2014',
   'Category': 'Beatles'},
  {'Name': 'Ringo',
   -1: 'Year',
   -9: '1.0',
   'Vehicles': 'Planes',
   -2: '2014',
   'Category': 'Beatles'},
  {'Name': 'Ringo',
   -1: 'Year',
   -9: '3.0',
   'Vehicles': 'Trai

In [9]:
import csv, io
from databaker.jupybakeutils import HLDUPgenerate_header_row

wdain = csv.reader(io.StringIO(Lx))

# First check that the headers are what we expect
wdaheaders = wdain.__next__()
numheaderadditionals = (len(wdaheaders) - len(template.headermeasurements))//len(template.headeradditionals)
if not (wdaheaders == HLDUPgenerate_header_row(numheaderadditionals)):
    print("WDA heades don't match.  nothing is likely to work now")
    
wdasegments = { }
for row in wdain:
    if row[0] == '*********':
        nrows = sum(map(len, wdasegments.values()))
        assert int(row[1]) == nrows, ("row number doesn't match", int(row[1]), nrows)
        assert len(list(wdain)) == 0
        break

    dval = { }
    isegmentnumber = None
    for r, k in zip(row, template.headermeasurements):
        if isinstance(k, tuple):
            nk = template.headermeasurementnumvalues[k[1]]
            if r:
                assert nk not in dval or dval[nk] == r
                dval[nk] = r
            else:
                assert not dval.get(nk)
        elif k == template.conversionsegmentnumbercolumn:
            isegmentnumber = int(r)
        else:
            assert not r
            
        lnumheaderadditionals = (len(row) - len(template.headermeasurements))
        assert lnumheaderadditionals % len(template.headeradditionals) == 0
        numheaderadditionals = lnumheaderadditionals//len(template.headeradditionals)
        for i in range(numheaderadditionals):
            rname, rvalue = None, None
            i0 = len(template.headermeasurements) + i*len(template.headeradditionals)
            for r, k in zip(row[i0:i0+len(template.headeradditionals)], template.headeradditionals):
                if isinstance(k, tuple):
                    if k[1] == "NAME":
                        assert rname is None or rname == r, (rname, r)
                        rname = r
                    else:
                        assert k[1] == "VALUE"
                        assert rvalue is None or rvalue == r
                        rvalue = r
                else:
                    assert not r
            assert rname, (rname, dval, row)
            dval[rname] = rvalue
    if isegmentnumber not in wdasegments:
        wdasegments[isegmentnumber] = [ ]
    wdasegments[isegmentnumber].append(dval)


In [22]:
help("asdasdasd".find)


Help on built-in function find:

find(...) method of builtins.str instance
    S.find(sub[, start[, end]]) -> int
    
    Return the lowest index in S where substring sub is found,
    such that sub is contained within S[start:end].  Optional
    arguments start and end are interpreted as in slice notation.
    
    Return -1 on failure.



In [27]:
help(io.StringIO)

Help on class StringIO in module io:

class StringIO(_TextIOBase)
 |  Text I/O implementation using an in-memory buffer.
 |  
 |  The initial_value argument sets the value of object.  The newline
 |  argument is like the one of TextIOWrapper's constructor.
 |  
 |  Method resolution order:
 |      StringIO
 |      _TextIOBase
 |      _IOBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getstate__(...)
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __setstate__(...)
 |  
 |  close(self, /)
 |      Close the IO object.
 |      
 |      Attempting any further operation after the object is closed
 |      will raise a ValueError.
 |      
 |      This method has no effect if the file is already closed.
 |  
